In [2]:
import numpy as np, os
from pathlib import Path

# Q7

(b)
- Using 0-indexing: ```[1, 2, 4, 6, 10, 12, 16, 18, 22, 26]```
- Using 1-indexing: ```[2, 3, 5, 7, 11, 13, 17, 19, 23, 27]```

In [3]:
with open("data/HW4/mystery.dat", "r") as f:
  lines = np.array([list(map(float, l.strip().split(","))) for l in f.readlines() if not l.isspace()])
print(lines.shape)

X = lines[:,:-1]
y = lines[:,[-1]]

print(X.shape, y.shape)

(101, 101)
(101, 100) (101, 1)


In [5]:
from sklearn import linear_model
import operator


for i in range(0, 40, 5):
  reg = linear_model.Ridge(alpha=i).fit(X, y)

  w, b = reg.coef_, reg.intercept_

  out = sorted(zip(list(w.flatten()), range(len(w.flatten()))), key = lambda x : -abs(x[0]))
  res = sorted(map(operator.itemgetter(1), out[:10]))

  print(f"Ridge, alpha={i}")
  print("\tOutput indexes: ", res)
  print("\tOutput indexes (1-indexed): ", [v + 1 for v in res])

  print("\tStrongest 10: ", np.round(np.array([v[0] for v in out[:10]]), 3))
  print("\tNext Strongest 5: ", np.round(np.array([v[0] for v in out[10:15]]), 3))
  print(f"\tGap: [{abs(out[9][0]) - abs(out[10][0])}]")

for i in np.linspace(0.01, 1, 10):
  reg = linear_model.Lasso(alpha=i).fit(X, y)

  w, b = reg.coef_, reg.intercept_

  out = sorted(zip(list(w.flatten()), range(len(w.flatten()))), key = lambda x : -abs(x[0]))
  res = sorted(map(operator.itemgetter(1), out[:10]))

  print(f"Lasso, alpha={i}")
  print("\tOutput indexes: ", res)
  print("\tOutput indexes (1-indexed): ", [v + 1 for v in res])

  print("\tStrongest 10: ", np.round(np.array([v[0] for v in out[:10]]), 3))
  print("\tNext Strongest 5: ", np.round(np.array([v[0] for v in out[10:15]]), 3))
  print(f"\tGap: [{abs(out[9][0]) - abs(out[10][0])}]")
  # for i, v in enumerate(out[:20]):
  #   if i == 10:
  #     print("VV== Should be sharp drop below ==VV")
  #   print(v)


Ridge, alpha=0
	Output indexes:  [1, 10, 20, 23, 26, 33, 43, 57, 67, 70]
	Output indexes (1-indexed):  [2, 11, 21, 24, 27, 34, 44, 58, 68, 71]
	Strongest 10:  [ 3.347  3.204  2.866 -2.501  2.248  2.22  -2.145  2.087 -2.073 -2.024]
	Next Strongest 5:  [ 1.922  1.889  1.859 -1.83  -1.8  ]
	Gap: [0.10115185856592723]
Ridge, alpha=5
	Output indexes:  [1, 2, 4, 6, 10, 12, 16, 18, 22, 26]
	Output indexes (1-indexed):  [2, 3, 5, 7, 11, 13, 17, 19, 23, 27]
	Strongest 10:  [1.094 1.073 0.936 0.928 0.839 0.824 0.815 0.777 0.737 0.644]
	Next Strongest 5:  [0.374 0.308 0.304 0.277 0.271]
	Gap: [0.27031900216990545]
Ridge, alpha=10
	Output indexes:  [1, 2, 4, 6, 10, 12, 16, 18, 22, 26]
	Output indexes (1-indexed):  [2, 3, 5, 7, 11, 13, 17, 19, 23, 27]
	Strongest 10:  [0.992 0.961 0.865 0.791 0.759 0.753 0.722 0.708 0.705 0.566]
	Next Strongest 5:  [ 0.355  0.264  0.241  0.237 -0.221]
	Gap: [0.21119022494312106]
Ridge, alpha=15
	Output indexes:  [1, 2, 4, 6, 10, 12, 16, 18, 22, 26]
	Output indexes (

# Q8

In [6]:
import pandas as pd

In [7]:
df = pd.read_csv("data/HW4/heart.csv")
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [8]:
X = np.array(df.iloc[:,:-1])
y = np.array(df.iloc[:,[-1]])

print(X.shape, y.shape)

(303, 13) (303, 1)


In [53]:
np.random.seed(123)
idx = np.random.choice(303, 303, replace=False)
tr_idx, te_idx = idx[:200], idx[200:]

X_tr, y_tr = X[tr_idx], y[tr_idx]
X_te, y_te = X[te_idx], y[te_idx]

def normalize(mode, X_tr, X_te):
  if mode == "z":
    # z-score normalize
    X_tr_mu = np.mean(X_tr, axis=0, keepdims=True)
    X_tr_sig = np.std(X_tr, axis=0, keepdims=True)

    X_tr_ = (X_tr - X_tr_mu) / X_tr_sig
    X_te_ = (X_te - X_tr_mu) / X_tr_sig
    return X_tr_, X_te_
  elif mode == "min-max":
    # min-max 0-1 normalization
    X_tr_mi = np.min(X_tr, axis=0, keepdims=True)
    X_tr_ma = np.max(X_tr, axis=0, keepdims=True)
    X_tr_ = (X_tr - X_tr_mi) / (X_tr_ma - X_tr_mi)
    X_te_ = (X_te - X_tr_mi) / (X_tr_ma - X_tr_mi)
    return X_tr_, X_te_
  else:
    return X_tr, X_te

print(X_tr.shape, y_tr.shape, X_te.shape, y_te.shape)

(200, 13) (200, 1) (103, 13) (103, 1)


# Q8a

The three most most significant features (based on magnitude of weight, after normalizing the features) are 'sex', 'ca', and 'cp' 

In [56]:
np.set_printoptions(linewidth=200)
col_names = list(df.columns)[:-1]

def most_influential(w):
  feats = sorted(list(zip(w.flatten(), col_names)), key = lambda x : -np.abs(x[0]))
  print(f"\t{[(v[1], np.round(v[0], 4)) for v in feats[:5]]}")

def test_err(reg: linear_model.LogisticRegression, X, y):
  y_hat = reg.predict(X)
  err = np.not_equal(y_hat, y.flatten())
  return np.sum(err)

def experiment(mode):
  print(f"*** Using {mode} Normalization")
  X_tr_, X_te_ = normalize(mode, X_tr, X_te)
  for C in [0]:
    print(f"L2 Regularization = {C}")
    if C == 0:
      reg = linear_model.LogisticRegression().fit(X_tr_, y_tr.flatten())
    else:
      reg = linear_model.LogisticRegression(C=C).fit(X_tr_, y_tr.flatten())
    w, b = reg.coef_, reg.intercept_
    print("\tw: ", np.round(w, 4).flatten())
    print("\tb: ", np.round(b, 4))
    most_influential(w)

    print(f"=== Testing === (Q8b)")
    err = test_err(reg, X_te_, y_te)  
    print(f"\tError Rate: {np.sum(err)}/{len(y_te)}={np.sum(err)/len(y_te):0.4f}")
  print()
  print("-"*50)
  print()

experiment("z")
experiment("min-max")
experiment(None)


*** Using z Normalization
L2 Regularization = 0
	w:  [-0.0366 -0.9783  0.8047 -0.4959 -0.3468  0.1503  0.1725  0.691  -0.4286 -0.7565  0.5111 -0.8314 -0.4907]
	b:  [-0.0234]
	[('sex', -0.9783), ('ca', -0.8314), ('cp', 0.8047), ('oldpeak', -0.7565), ('thalach', 0.691)]
=== Testing === (Q8b)
	Error Rate: 13/103=0.1262

--------------------------------------------------

*** Using min-max Normalization
L2 Regularization = 0
	w:  [-0.6315 -1.3791  1.6426 -1.1085 -0.6228  0.1735  0.5641  1.4912 -0.867  -1.7894  1.3386 -1.8582 -1.3364]
	b:  [1.4089]
	[('ca', -1.8582), ('oldpeak', -1.7894), ('cp', 1.6426), ('thalach', 1.4912), ('sex', -1.3791)]
=== Testing === (Q8b)
	Error Rate: 13/103=0.1262

--------------------------------------------------

*** Using None Normalization
L2 Regularization = 0
	w:  [ 0.0184 -1.5697  0.7578 -0.0242 -0.0055  0.1327  0.4142  0.0374 -0.6592 -0.6019  0.7472 -0.7584 -0.7095]
	b:  [0.0501]
	[('sex', -1.5697), ('ca', -0.7584), ('cp', 0.7578), ('slope', 0.7472), ('th

c:\Users\James\anaconda3\envs\mlenv\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


# Q8c

In [59]:
n = len(X_tr)
folds = 5
stats_rate = []
stats_wrong = []

def k_exp(mode):
  print(f"=== Mode: {mode} ===")
  for fold in range(folds):
    b1 = np.round(n / folds * fold).astype(int)
    b2 = np.round(n / folds * (fold + 1)).astype(int)
    X_trr = np.vstack((X_tr[:b1,:], X_tr[b2:,:]))
    y_trr = np.vstack((y_tr[:b1,:], y_tr[b2:,:]))
    X_va = X_tr[b1:b2]
    y_va = y_tr[b1:b2]

    X_tr_, X_va_ = normalize(mode, X_trr, X_va)

    reg = linear_model.LogisticRegression().fit(X_tr_, y_trr.flatten())
    err = test_err(reg, X_va_, y_va)

    print(f"\t[Fold {fold}] Error Rate: {np.sum(err)}/{len(y_va)}={np.sum(err)/len(y_va):0.4f}")
    stats_rate.append(err / len(y_va))
    stats_wrong.append(err)
  print(f"sum(rate)/5 = {np.mean(stats_rate).item():0.4f}")
  print(f"sum(wrong)/5 = {np.mean(stats_wrong).item():0.4f}")
k_exp("z")
k_exp("min-max")
k_exp(None)
  

=== Mode: z ===
	[Fold 0] Error Rate: 9/40=0.2250
	[Fold 1] Error Rate: 8/40=0.2000
	[Fold 2] Error Rate: 10/40=0.2500
	[Fold 3] Error Rate: 10/40=0.2500
	[Fold 4] Error Rate: 4/40=0.1000
sum(rate)/5 = 0.2050
sum(wrong)/5 = 8.2000
=== Mode: min-max ===
	[Fold 0] Error Rate: 9/40=0.2250
	[Fold 1] Error Rate: 9/40=0.2250
	[Fold 2] Error Rate: 10/40=0.2500
	[Fold 3] Error Rate: 8/40=0.2000
	[Fold 4] Error Rate: 2/40=0.0500
sum(rate)/5 = 0.1975
sum(wrong)/5 = 7.9000
=== Mode: None ===
	[Fold 0] Error Rate: 10/40=0.2500
	[Fold 1] Error Rate: 7/40=0.1750
	[Fold 2] Error Rate: 9/40=0.2250
	[Fold 3] Error Rate: 9/40=0.2250
	[Fold 4] Error Rate: 3/40=0.0750
sum(rate)/5 = 0.1950
sum(wrong)/5 = 7.8000


c:\Users\James\anaconda3\envs\mlenv\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\James\anaconda3\envs\mlenv\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logisti